In [2]:
import numpy as np
import pandas as pd 
import seaborn as sns

In [1]:
"""
Заполнение пропусков: 

Начнем заполить с помощью средней (статистическая характеристика )
Но заполнение не просто так, а деления на страты.
Будем делить на страты не зная, есть они или нет
Так как страты нам неизвестны, мы возьмем алогоритм кластеризации 
Чтобы оценить мы придумываем искусственную метрику


"""

'\nЗаполнение пропусков: \n\nНачнем заполить с помощью средней (статистическая характеристика )\nНо заполнение не просто так, а деления на страты. \nБудем делить наши признаки на страты, не \nЧтобы оценить мы придумываем искусственную метрику\n\n\n'

In [6]:
from sklearn.cluster import KMeans #Алгоритм кластеризации, чтобы делить наши данные на страты
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import trange, tqdm

In [5]:
from sklearn.datasets import load_diabetes #датасет диабетиков

In [7]:
load_diabetes().keys()

dict_keys(['data', 'target', 'frame', 'DESCR', 'feature_names', 'data_filename', 'target_filename'])

In [8]:
#данные уже стандартизированы (мат ожидание к 0, а дисперсия к 1). Поэтому, к примеру. пол такой 
data = load_diabetes()
feature_names = data.feature_names
df = pd.DataFrame(data.data, columns = feature_names)
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
dtypes: float64(10)
memory usage: 34.7 KB


In [ ]:
# Наш датасет не содеражит пропусков. Нам нужно их создасть 
prop_null = 0.1  0.1 #определеим вероятность того, что значение пропущено 10%
pr = 'bmi'  #Выберем колонку,  в которой будут пропущенные значения 

data = df.copy()  #Копируем датасет, чтобы можно было сравнить с исходником
data.loc[np.random.randint(0,df.shape[0],int(df.shape[0]*prop_null//1)),pr] = None

#Берем определенное кол-во чисел np.random.randint ( от 0 до размера датасета, а кол-во берем, 
#равное кол-ву во всем датасете умноженное на нашу вероятность 10% деление на 1 сделано для того, чтобюы
#к целому числу потом было легко привести
#pr колонка, которую мы выбрали, где будут пропуски 
#То есть мы выбрали 10% значний признака из выборки, выдернули, и туда поставили наны


ind = data[data[pr].isna()].index 
#Сохраним индексы пропущенных значений

cols = list(data.select_dtypes([np.number]).columns) 
#Возьмем все колонки, которые имеют тип численный
cols.remove(pr)
#Из этих колонок уберем нашу исходную, тк мы не хотим, чтобы она участвовала
#Это делаем для того, чтобы разделить наши данные на страты
#В итоге у нас будет два числа, которые будут показывать, насколько хорошо мы заполнили наши данные
#Ср. ариф. ошибка и дисперсия. Чем меньше эти два числа, тем лучше.

delta_ = np.array(data.fillna(data[pr].mean()).loc[ind, pr] - df.loc[ind, pr])
#Посчитаем разницу между истинными значениями и подставим среднюю 
#т.е возьмем нашу исходную мтарицу, возьмем в которой заполнили пропуски и возьмем разницу и в пропущенных значениях 
#будет разница отличная от 0
#data.fillna(data[pr].mean() запоняем наши данные просто ср. значением по всей выборке
#loc[ind, pr] берем индексы, которые записали ранее и сам признак
#df.loc[ind, pr] потом вычитаем из исходного датасета и получаем разницу


print('start: ',delta_.mean(),delta_.std())
#Берем  ср. и дисперсию 

#Кластеризуем
kmeans = KMeans(n_clusters=1).fit(data.loc[~data.index.isin(ind),cols])
#наши данные кластеризуем на 1 кластер KMeans(n_clusters=1)
#в качестве обучения передаем наши данные fit(data.loc[~data.index.isin(ind),cols]), то есть те семплы, которые не попадают в наши индексы

data['cl'] = kmeans.predict(data[cols])
#мы получаем обученный kmeans и с помощью него предсказываем наши данные и добавляем колонку, где хранится то значение кластера, 
#и то значение старты, потенциального на которое поделили


#Заполним наши пропуски 
for i in ind:
    data.at[i, pr] = data[data['cl'] == data.at[i,'cl']][pr].mean()
#Из наших данных выбираем те, которые пренадлежат нашим индексам 

delta = np.abs(np.array(data.loc[ind, pr] - df.loc[ind, pr]))
print('finish: ',delta.mean(),delta.std())     

start:  0.002926763472746897 0.04569703448706017
finish:  0.0375292147985165 0.02623629055228237


In [ ]:
_=[print(f" {x:4.3f} -> {y:4.3f}  \t{'+' if np.abs(x) > np.abs(y) else '-'}") for x,y in zip(delta_,(df - data).loc[ind,pr])]

 0.026 -> -0.001  	+
 0.058 -> -0.031  	+
 -0.010 -> 0.037  	-
 0.024 -> 0.001  	+
 0.005 -> 0.022  	-
 0.021 -> -0.047  	-
 -0.009 -> 0.000  	+
 0.006 -> -0.015  	-
 -0.058 -> 0.049  	+
 -0.023 -> 0.048  	-
 0.071 -> -0.044  	+
 -0.047 -> 0.021  	+
 0.034 -> -0.043  	-
 0.066 -> -0.040  	+
 0.061 -> -0.034  	+
 -0.059 -> 0.033  	+
 -0.009 -> 0.000  	+
 -0.042 -> 0.015  	+
 0.055 -> -0.029  	+
 0.013 -> -0.039  	-
 0.047 -> -0.022  	+
 -0.006 -> -0.003  	+
 0.040 -> -0.014  	+
 0.025 -> -0.034  	-
 -0.018 -> -0.008  	+
 -0.092 -> 0.119  	-
 0.032 -> -0.041  	-
 0.025 -> -0.034  	-
 0.040 -> -0.049  	-
 -0.004 -> 0.029  	-
 0.008 -> -0.034  	-
 -0.053 -> 0.027  	+
 0.057 -> -0.066  	-
 -0.021 -> 0.012  	+
 -0.017 -> -0.009  	+
 0.025 -> -0.034  	-
 -0.002 -> -0.024  	-
 -0.001 -> 0.027  	-
 0.071 -> -0.044  	+
 -0.053 -> 0.078  	-
 0.061 -> -0.036  	+
 -0.001 -> 0.026  	-
